In [6]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import json
import kagglehub
import os

os.environ["KAGGLEHUB_CACHE"] = "./data/job_offer/"

In [7]:
DL = kagglehub.dataset_download("ravindrasinghrana/job-description-dataset")
path = DL + "/job_descriptions.csv"
print("Path to dataset files:", path)
df = pd.read_csv(path)
df = df.sample(10000)

fields_to_combine = ["Job Id", "Job Title", "Job Description", "skills", "Responsibilities", "Company", "location"]
df["combined_text"] = df[fields_to_combine].astype(str).agg(" ".join, axis=1)

100%|██████████| 457M/457M [01:09<00:00, 6.89MB/s] 

Extracting files...


Path to dataset files: ./data/job_offer/datasets/ravindrasinghrana/job-description-dataset/versions/1/job_descriptions.csv


In [5]:
model = SentenceTransformer("all-MiniLM-L6-v2")
job_embeddings = model.encode(df["combined_text"].tolist(), convert_to_numpy=True, show_progress_bar=True, normalize_embeddings=True)

faiss_index = faiss.IndexFlatIP(job_embeddings.shape[1])
faiss_index.add(job_embeddings)
faiss.write_index(faiss_index, "./data/jobs_index.faiss")

job_id_mapping = {i: jid for i, jid in enumerate(df["Job Id"].tolist())}
with open("./data/jobs_index_mapping.json", "w", encoding="utf-8") as f:
    json.dump(job_id_mapping, f, indent=4)

Batches:   0%|          | 0/313 [00:00<?, ?it/s]